# Milestone 2 - CasierVert952

This is a part of the [analysis.ipynb](analysis.ipynb) that performs the preprocessing of the data from two beers ratings websites (BeerAdvocate and RateBeer).

## Preprocessing

### 1 Importation of libraries

In [1]:
# Import the basic requiered libraries
import os
import csv
import time
import numpy as np
import pandas as pd

# Path variables
BA_DATA_PATH = "data/BeerAdvocate/"
RB_DATA_PATH = "data/RateBeer/"

### 2 Transformation of TXT ratings files to CSV

You can download the ```ratings.csv``` files for both dataset with the following links (~2GB each):

- For BA : [here](https://coursedingler.ch/data/BA/ratings.csv)
- For RB : [here](https://coursedingler.ch/data/RB/ratings.csv)

The following cell should **NOT** be executed, it only shows how the ```ratings.csv``` for each dataset were generated.

It take around 19 minutes to generate the BA ratings file and 14 minutes for the RB one.

```python
from helpers import txt_to_csv

file_txt = 'ratings.txt'
file_csv = 'ratings.csv'

txt_to_csv(BA_DATA_PATH + file_txt, BA_data_path + file_csv, "BA")
txt_to_csv(RB_DATA_PATH + file_txt, RB_data_path + file_csv, "RB")
```

**Make sure you have been placed or generated the ```ratings.csv``` files in the ```BeerAdvocate``` and ```RateBeer``` folders as well as all other data files when executing the following cells !**
```
data/
├── BeerAdvocate
│   ├── beers.csv
│   ├── breweries.csv
│   ├── users.csv
│   └── ratings.csv
│
└── RateBeer
    ├── beers.csv
    ├── breweries.csv
    ├── users.csv
    └── ratings.csv
```



### 3 Loading CSV data
Let first import the data in CSV format for the two datasets, the users, the beers and the breweries.

In [2]:
# Create Dataframes for the BA dataset
BA_beers = pd.read_csv(BA_DATA_PATH + 'beers.csv')
BA_breweries = pd.read_csv(BA_DATA_PATH + 'breweries.csv')
BA_users = pd.read_csv(BA_DATA_PATH + 'users.csv')

# Create Dataframes for the RB dataset
RB_beers = pd.read_csv(RB_DATA_PATH + 'beers.csv')
RB_breweries = pd.read_csv(RB_DATA_PATH + 'breweries.csv')
RB_users = pd.read_csv(RB_DATA_PATH + 'users.csv')

In [3]:
# Create Dataframes for the BA ratings file
s_time = time.time()
BA_rating = pd.read_csv(BA_DATA_PATH + 'ratings.csv')
e_time = time.time()
print("Reading of BA ratings ended in " + str(e_time - s_time) + " seconds.")

# Create Dataframes for the RB ratings file
s_time = time.time()
RB_rating = pd.read_csv(RB_DATA_PATH + 'ratings.csv')
e_time = time.time()
print("Reading of RB ratings ended in " + str(e_time - s_time) + " seconds.")

FileNotFoundError: [Errno 2] No such file or directory: 'data/BeerAdvocate/ratings.csv'

### 4 Merging data

#### 4.1 Dropping and renaming columns

We are dropping the columns that are not needed, some of them will be recovered during the merging phase. The columns are renamed to avoid colisions during the merges.

In [4]:
from helpers import ratings_dict

# Removing not needed columns
BA_rating.drop(columns=["text", "review"], inplace=True)
RB_rating.drop(columns=["text"], inplace=True)

# Removing columns that will be recovered when merging
BA_rating.drop(columns=["brewery_name", "style", "beer_name", "user_name", "abv"], inplace=True)
RB_rating.drop(columns=["brewery_name", "style", "beer_name", "user_name", "abv"], inplace=True)

# Renaming the columns as define by "ratings_dict"
BA_rating.rename(columns=ratings_dict, inplace=True)
RB_rating.rename(columns=ratings_dict, inplace=True)

#### 4.2 Merging with beers 's Data

In [5]:
from helpers import beers_dict

# Merging with the beers's data
BA_merged = pd.merge(BA_rating, BA_beers, on=["beer_id", "brewery_id"], how="inner")
RB_merged = pd.merge(RB_rating, RB_beers, on=["beer_id", "brewery_id"], how="inner")

# Renaming the columns as define by "beers_dict"
BA_merged.rename(columns=beers_dict, inplace=True)
RB_merged.rename(columns=beers_dict, inplace=True)

#### 4.3 Merging with breweries's data

In [6]:
from helpers import breweries_dict

# Merging with the breweries's data
BA_merged = pd.merge(BA_merged, BA_breweries, left_on="brewery_id", right_on="id", how="inner")
RB_merged = pd.merge(RB_merged, RB_breweries, left_on="brewery_id", right_on="id", how="inner")

# Dropping the duplicate columns
BA_merged.drop(columns=["id", "name"], inplace=True)
RB_merged.drop(columns=["id", "name"], inplace=True)

# Renaming the columns as define by "breweries_dict"
BA_merged.rename(columns=breweries_dict, inplace=True)
RB_merged.rename(columns=breweries_dict, inplace=True)

#### 4.4 Merging with users's data

In [7]:
from helpers import users_dict

# Merging with the users's data
BA_merged = pd.merge(BA_merged, BA_users, on=["user_id"], how="inner")
RB_merged = pd.merge(RB_merged, RB_users, on=["user_id"], how="inner")

# Renaming the columns as define by "users_dict"
BA_merged.rename(columns=users_dict, inplace=True)
RB_merged.rename(columns=users_dict, inplace=True)

### 5 Data cleaning

#### 5.1 Changing dates format

The format of the dates fields were initialy the timestamp format, here we convert it in a human readable format. We only keep the month and the year as it's the only element we need for our analyses.

In [8]:
# Changing the format of the rating date to the format "Month-Year"
BA_cleaned = BA_merged
BA_cleaned["rating_date"] = pd.to_datetime(BA_cleaned["rating_date"], unit='s').dt.strftime("%m-%Y")

RB_cleaned = RB_merged
RB_cleaned["rating_date"] = pd.to_datetime(RB_cleaned["rating_date"], unit='s').dt.strftime("%m-%Y")

# Changing the format of the user_join date to the format "Month-Year"
BA_cleaned = BA_merged
BA_cleaned["user_join_date"] = pd.to_datetime(BA_cleaned["user_join_date"], unit='s').dt.strftime("%m-%Y")

RB_cleaned = RB_merged
RB_cleaned["user_join_date"] = pd.to_datetime(RB_cleaned["user_join_date"], unit='s').dt.strftime("%m-%Y")

#### 5.2 Extracting the country and states

Since our analyses will focus on american states, we have to divide the user location field in ```country``` and ```state```. The state is filled with ```nan``` if not present.

In [9]:
# Spliting the "user_location" in country and state fields
split_locations = BA_cleaned['user_location'].str.split(',', expand=True)
BA_cleaned['user_country'] = split_locations[0].str.strip()
BA_cleaned['user_state'] = split_locations[1].str.strip() if len(split_locations) > 1 else np.nan

split_locations = RB_cleaned['user_location'].str.split(',', expand=True)
RB_cleaned['user_country'] = split_locations[0].str.strip()
RB_cleaned['user_state'] = split_locations[1].str.strip() if len(split_locations) > 1 else np.nan

We divide breweries location field too.

In [10]:
# Spliting the "breweries_location" in country and state fields
split_locations = BA_cleaned['breweries_location'].str.split(',', expand=True)
BA_cleaned['breweries_country'] = split_locations[0].str.strip()
BA_cleaned['breweries_state'] = split_locations[1].str.strip() if len(split_locations) > 1 else np.nan

split_locations = RB_cleaned['breweries_location'].str.split(',', expand=True)
RB_cleaned['breweries_country'] = split_locations[0].str.strip()
RB_cleaned['breweries_state'] = split_locations[1].str.strip() if len(split_locations) > 1 else np.nan

Now just changing the order of the columns for a better visibility.

In [13]:
BA_new_column_order = [
    'rating_date', 'rating_appearance',
    'rating_aroma', 'rating_palate', 'rating_taste', 'rating_overall',
    'rating', 'beer_id', 'beer_name', 'beer_style', 'beer_nbr_ratings',
    'beer_nbr_reviews', 'beer_avg', 'beer_ba_score', 'beer_bros_score',
    'beer_abv', 'beer_avg_computed', 'beer_zscore',
    'beer_nbr_matched_valid_ratings', 'beer_avg_matched_valid_ratings', 'brewery_id',
    'brewery_name', 'breweries_location', 'breweries_country', 'breweries_state', 'breweries_nbr_beers',
    'user_id', 'user_nbr_ratings', 'user_nbr_reviews', 'user_name', 'user_join_date',
    'user_location', 'user_country', 'user_state'
]

RB_new_column_order = [ 
    'rating_date', 'rating_appearance',
    'rating_aroma', 'rating_palate', 'rating_taste', 'rating_overall',
    'rating', 'beer_id', 'beer_name', 'beer_style', 'beer_nbr_ratings',
    'overall_score', 'style_score', 'beer_avg', 'beer_abv',
    'beer_avg_computed', 'beer_zscore', 'beer_nbr_matched_valid_ratings',
    'beer_avg_matched_valid_ratings', 'brewery_id', 'brewery_name', 'breweries_location',
    'breweries_country', 'breweries_state', 'breweries_nbr_beers', 'user_id', 'user_nbr_ratings', 'user_name',
    'user_join_date', 'user_location', 'user_country', 'user_state'
]
BA_cleaned = BA_cleaned[BA_new_column_order]
RB_cleaned = RB_cleaned[RB_new_column_order]

### 6 Saving cleaned data

The following cell will save the data merged and cleaned in CSV file. After that step, the two dataframes can be loaded quickly without performing the preprocessing step again and again.

In [12]:
# Saving the cleaned data in CSV files
BA_cleaned.to_csv(BA_DATA_PATH + 'BA_cleaned.csv', index=False)
RB_cleaned.to_csv(RB_DATA_PATH + 'RB_cleaned.csv', index=False)